# Decision Tree Lab

In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
import numpy as np
from scipy.io import arff
import matplotlib.pyplot as plt
import pandas as pd
import json
from random import seed
from random import randrange
from sklearn.datasets import load_iris

## 1. Implement the ID3 decision tree algorithm  
- Use standard information gain as your basic attribute evaluation metric.  Note that ID3 would usually augment information gain with a mechanism to penalize statistically insignificant attribute splits to avoid overfit (e.g. early stopping, gain ratio, etc.)
- Include the ability to handle unknown attributes by making "unknown" a separate output class.
- You do not need to handle real valued attributes.
- You are welcome to create other classes and/or functions in addition to the ones provided below. (e.g. If you build out a tree structure, you might create a node class).
- It is a good idea to use simple data sets (like the lenses data and the pizza homework), which you can check by hand, to test each detailed step of your algorithm to make sure it works correctly. 

In [2]:
class DTClassifier(BaseEstimator,ClassifierMixin):

    def __init__(self,counts=None):
        """ Initialize class with chosen hyperparameters.
        Args:
        Optional Args (Args we think will make your life easier):
            counts: A list of Ints that tell you how many types of each feature there are
        Example:
            DT  = DTClassifier()
            or
            DT = DTClassifier(count = [2,3,2,2])
            Dataset = 
            [[0,1,0,0],
            [1,2,1,1],
            [0,1,1,0],
            [1,2,0,1],
            [0,0,1,1]]

        """
        self.ig_splits = []
        

    def fit(self, x, y, Print=False):
        """ Fit the data; Make the Decision tree

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 1D numpy array with the training targets

        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)

        """
        data = x.copy()
        data[y.name] = y
        self.tree = self.convert(self.decision_tree(data, data, x.columns, y.name, Print))


    def predict(self, X):
        """ Predict all classes for a dataset X

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets

        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """
        
        # convert input data into a dictionary of samples
        samples = self.convert(X.to_dict(orient='records'))
        
        predictions = []
        # make a prediction for every sample
        for sample in samples:
          predictions.append(self.make_prediction(sample, self.tree, 1.0))
        return predictions

    def entropy(self, attribute_column):
        # find unique values and their frequency counts for the given attribute
        values, counts = np.unique(attribute_column, return_counts=True)

        # calculate entropy for each unique value
        entropy_list = []

        for i in range(len(values)):
          probability = counts[i]/np.sum(counts)
          entropy_list.append(-probability*np.log2(probability))

        # calculate sum of individual entropy values
        total_entropy = np.sum(entropy_list)

        return total_entropy
    
    def information_gain(self, data, feature_attribute_name, target_attribute_name):
        # find total entropy of given subset
        total_entropy = self.entropy(data[target_attribute_name])

        # find unique values and their frequency counts for the attribute to be split
        values, counts = np.unique(data[feature_attribute_name], return_counts=True)

        # calculate weighted entropy of subset
        weighted_entropy_list = []

        for i in range(len(values)):
          subset_probability = counts[i]/np.sum(counts)
          subset_entropy = self.entropy(data.where(data[feature_attribute_name]==values[i]).dropna()[target_attribute_name])
          weighted_entropy_list.append(subset_probability*subset_entropy)

        total_weighted_entropy = np.sum(weighted_entropy_list)

        # calculate information gain
        information_gain = total_entropy - total_weighted_entropy

        return information_gain
    
    def decision_tree(self, data, orginal_data, feature_attribute_names, target_attribute_name,
                      Print=False, depth="", parent_node_class=None):
        # base cases:
        # if data is pure, return the majority class of subset
        unique_classes = np.unique(data[target_attribute_name])
        if len(unique_classes) <= 1:
            if(Print):
                print(f"{depth}prediction = {unique_classes[0].decode()}")
            return unique_classes[0]
        # if subset is empty, ie. no samples, return majority class of original data
        elif len(data) == 0:
          majority_class_index = np.argmax(np.unique(original_data[target_attribute_name], return_counts=True)[1])
          return np.unique(original_data[target_attribute_name])[majority_class_index]
        # if data set contains no features to train with, return parent node class
        elif len(feature_attribute_names) == 0:
          return parent_node_class
        # if none of the above are true, construct a branch:
        else:
          # determine parent node class of current branch
          majority_class_index = np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])
          parent_node_class = unique_classes[majority_class_index]

          # determine information gain values for each feature
          # choose feature which best splits the data, ie. highest value
          ig_values = [self.information_gain(data, feature, target_attribute_name) for feature in feature_attribute_names]
          best_feature_index = np.argmax(ig_values)
          self.ig_splits.append(ig_values[best_feature_index])
          best_feature = feature_attribute_names[best_feature_index]
            
          # create tree structure, empty at first
          tree = {best_feature: {}}

          # remove best feature from available features, it will become the parent node
          feature_attribute_names = [i for i in feature_attribute_names if i != best_feature]

          # create nodes under parent node
          parent_attribute_values = np.unique(data[best_feature])
          for value in parent_attribute_values:
            sub_data = data.where(data[best_feature] == value).dropna()
            if Print:
                print(f"{depth}{best_feature} = {value.decode()}")
            # call the algorithm recursively
            subtree = self.decision_tree(sub_data, orginal_data, feature_attribute_names,
                                         target_attribute_name, Print, depth+'    ', parent_node_class)

            # add subtree to original tree
            tree[best_feature][value] = subtree

          return tree
    
    def make_prediction(self, sample, tree, default=1):
        # map sample data to tree
        for attribute in list(sample.keys()):
          # check if feature exists in tree
          if attribute in list(tree.keys()):
            try:
                result = tree[attribute][sample[attribute]]
            except:
                return default

            result = tree[attribute][sample[attribute]]

            # if more attributes exist within result, recursively find best result
            if isinstance(result, dict):
                return self.make_prediction(sample, result)
            else:
                return result
    
    def score(self, X, y):
        """ Return accuracy(Classification Acc) of model on a given dataset. Must implement own score function.

        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 1D numpy array of the targets 
        """
        pred = self.predict(X)
        y = np.array(self.convert(list(y)))
        return np.mean(pred == y)
    
    def convert(self, data):
        if isinstance(data, bytes):  return data.decode()
        if isinstance(data, dict):   return dict(map(self.convert, data.items()))
        if isinstance(data, tuple):  return tuple(map(self.convert, data))
        if isinstance(data, list):   return list(map(self.convert, data))
        return data



### 1.1 (20%) Debug 

- Debug your model by training on the lenses dataset: [Debug Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/lenses.arff)
- Test your model on the lenses test set: [Debug Test Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/lenses_test.arff)
- Parameters:
For this problem the number of unique feature values for each feature is: counts = [3,2,2,2] (You should compute this when you read in the data, before fitting)
---

Expected Results: Accuracy = [0.33]

Information gain at splits = [0.5487949406953987, 0.7704260414863775, 0.3166890883150208, 1.0, 0.4591479170272447, 0.9182958340544894]

Predictions should match this file: [Lenses Predictions](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/pred_lenses.csv)

*NOTE: The [Lenses Prediction](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/pred_lenses.csv) uses the following encoding: soft=2, hard=0, none=1. Use this same encoding.*

<!-- You should be able to get about 68% (61%-82%) predictive accuracy on the lenses data -->

Here's what your decision tree splits and information gains should look like, with the corresponding child node predictions:

<pre>
tear_prod_rate = normal: 0.5487949406953987
    astigmatism = no: 0.7704260414863775
        age = pre_presbyopic: 0.3166890883150208
            prediction: soft
        age = presbyopic:
            spectacle_prescrip = hypermetrope: 1.0
                prediction: soft
            spectacle_prescrip = myope:
                prediction: none
        age = young:
            prediction: soft
    astigmatism = yes:
        spectacle_prescrip = hypermetrope: 0.4591479170272447
            age = pre_presbyopic: 0.9182958340544894
                prediction: none
            age = presbyopic:
                prediction: none
            age = young:
                prediction: hard
        spectacle_prescrip = myope:
            prediction: hard
tear_prod_rate = reduced:
    prediction: none
</pre>

In [3]:
data = pd.DataFrame(arff.loadarff('lenses.arff')[0], dtype=float)

# Train Decision Tree
x_train = data.drop(columns="contact_lenses")
y_train = data["contact_lenses"]
dt = DTClassifier()
dt.fit(x_train, y_train, True)
print(dt.ig_splits)

# Load debug test data
data = pd.DataFrame(arff.loadarff('lenses_test.arff')[0], dtype=float)

# Predict and compute model accuracy
x_test = data.drop(columns="contact_lenses")
y_test = data["contact_lenses"]
dt.score(x_test, y_test)


tear_prod_rate = normal
    astigmatism = no
        age = pre_presbyopic
            prediction = soft
        age = presbyopic
            spectacle_prescrip = hypermetrope
                prediction = soft
            spectacle_prescrip = myope
                prediction = none
        age = young
            prediction = soft
    astigmatism = yes
        spectacle_prescrip = hypermetrope
            age = pre_presbyopic
                prediction = none
            age = presbyopic
                prediction = none
            age = young
                prediction = hard
        spectacle_prescrip = myope
            prediction = hard
tear_prod_rate = reduced
    prediction = none
[0.5487949406953982, 0.7704260414863778, 0.3166890883150208, 1.0, 0.4591479170272448, 0.9182958340544896]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  # This is added back by InteractiveShellApp.init_path()


0.3333333333333333

My accuracy and information gain at splits came out correctly and worked as expected as well as the tree splits.

In [4]:
# Optional Debugging Dataset - Pizza Homework
# pizza_dataset = np.array([[1,2,0],[0,0,0],[0,1,1],[1,1,1],[1,0,0],[1,0,1],[0,2,1],[1,0,0],[0,2,0]])
# pizza_labels = np.array([2,0,1,2,1,2,1,1,0])

### 1.2 (20%) Evaluation 

- We will evaluate your model based on its performance on the zoo dataset. 
- Train your model using this dataset: [Evaluation Train Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/zoo.arff)
- Test your model on this dataset: [Evaluation Test Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/zoo_test.arff)
- Parameters: counts = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2] (You should compute this when you read in the data, before fitting)
---
Your progam should print out your accuracy on the evaluation test dataset and also the information gain of each split you make.

In [5]:
# Load evaluation training data
data = pd.DataFrame(arff.loadarff('zoo.arff')[0])

# Train Decision Tree
x_train = data.drop(columns="type")
y_train = data["type"]
dt = DTClassifier()
dt.fit(x_train, y_train, True)
print(dt.ig_splits)

# Load evaluation test data
data = pd.DataFrame(arff.loadarff('zoo_test.arff')[0])

# Predict and compute model accuracy
x_test = data.drop(columns="type")
y_test = data["type"]
dt.score(x_test, y_test)

legs = 0
    fins = F
        toothed = F
            prediction = c7
        toothed = T
            prediction = c3
    fins = T
        eggs = F
            prediction = cT
        eggs = T
            prediction = c4
legs = 2
    hair = F
        prediction = c2
    hair = T
        prediction = cT
legs = 4
    hair = F
        predator = F
            prediction = c3
        predator = T
            toothed = F
                prediction = c7
            toothed = T
                prediction = c5
    hair = T
        prediction = cT
legs = 5
    prediction = c7
legs = 6
    predator = F
        prediction = c6
    predator = T
        prediction = c7
legs = 8
    prediction = c7
[1.3630469031539394, 0.8865408928220899, 0.9852281360342515, 0.6962122601251459, 0.8256265261578954, 0.6892019851173656, 0.8631205685666308, 0.7219280948873623, 0.7219280948873623]


0.147

In the Zoo dataset, the information gain at the first split was very high at 1.36 the other splits after were not as high. The total model accuracy of the dataset was .147 which was about half of what we got from the lenses dataset. 

## 2. Learn Cars and Voting Data Sets and Predict accuracy with *n*-fold CV  
- Use your ID3 algorithm to induce decision trees for the cars dataset and the voting dataset.  Do not use a stopping criteria, but induce the tree as far as it can go (until classes are pure or there are no more data or attributes to split on).
- Implement and use 10-fold Cross Validation (CV) on each data set to predict how well the models will do on novel data.  
- For each dataset, create a table with the training, validation, and test classification accuracy for each of the 10 runs and the average accuracies for the training, validation, and test data. 
- As a rough sanity check, typical decision tree accuracies for these data sets are: Cars: .90-.95, Vote: .92-.95.

### 2.1 (15%) Implement 10-fold Cross Validation and report results for the Cars Dataset
- Use this [Cars Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/cars.arff)
- Create a table for your *n*-fold cross validation accuracies

*If you are having trouble using scipy's loadarff function (scipy.io.arff.loadarff), try:*

*pip install arff &nbsp;&nbsp;&nbsp;&nbsp;          # Install arff library*

*import arff as arf*                   

*cars = list(arf.load('cars.arff'))   &nbsp;&nbsp;&nbsp;&nbsp;# Load your downloaded dataset (!curl, etc.)*

*df = pd.DataFrame(cars)*  

*There may be additional cleaning needed*

In [14]:
def fold_i_of_k(dataset, i, k):
    n = len(dataset)
    return dataset[n*(i-1)//k:n*i//k]

def k_fold_cross_validate(data, output_class, folds=3):
    data = data.sample(frac=1).reset_index(drop=True)
    data_set = []
    scores = []
    j = 0
    for i in range(1, folds+1):
        data_set.append(fold_i_of_k(data, i, folds))
    for i in range(folds):
        dfs_1 = data_set[0:i]
        dfs_2 = data_set[i+1:len(data_set)+1]
        df = dfs_1 + dfs_2
        df = pd.concat(df)
        x_train = df.drop(columns=output_class)
        y_train = df[output_class]
        x_test = data_set[i].drop(columns=output_class)
        y_test = data_set[i][output_class]
        dt = DTClassifier()
        dt.fit(x_train, y_train)
        score = dt.score(x_test, y_test)
        print(score)
        scores.append(score)
    return scores



data = arff.loadarff('cars.arff')[0]
df =  pd.DataFrame(data)
scores = k_fold_cross_validate(df, "class", 10)
print(f"average accuracy: {np.mean(np.mean(scores))}")
cdt = dt.tree

data = pd.DataFrame(arff.loadarff('cars.arff')[0])
x_train = data.drop(columns="class")
y_train = data["class"]
dt = DTClassifier()
dt.fit(x_train, y_train, True)
print(dt.ig_splits)

# Report Average Test Accuracy

0.8837209302325582
0.9075144508670521
0.8728323699421965
0.8959537572254336
0.884393063583815
0.8837209302325582
0.8786127167630058
0.9075144508670521
0.8901734104046243
0.884393063583815
average accuracy: 0.888882914370211
safety = high
    persons = 2
        prediction = unacc
    persons = 4
        buying = high
            maint = high
                prediction = acc
            maint = low
                prediction = acc
            maint = med
                prediction = acc
            maint = vhigh
                prediction = unacc
        buying = low
            maint = high
                lug_boot = big
                    prediction = vgood
                lug_boot = med
                    doors = 2
                        prediction = acc
                    doors = 3
                        prediction = acc
                    doors = 4
                        prediction = vgood
                    doors = 5more
                        prediction = vgood
         

My average accuracy for the 10 fold cross validation was about .88 or 88% with the cars dataset. My decision tree was .01 away from getting the typical. As stated above the accuracy should be .90 - .95.

### 2.3 (15%) Voting Dataset 
- Use this [Voting Dataset with missing values](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/voting_with_missing.arff)
- Create a table for your *n*-fold cross validation accuracies
- This data set has don't know data.  Discuss how your algorithm handles this

In [15]:
data = arff.loadarff('voting_with_missing.arff')[0]
df =  pd.DataFrame(data)
df = df.replace("?", "unknown")
scores = k_fold_cross_validate(df, "Class", 10)
print(f"average accuracy: {np.mean(np.mean(scores))}")

data = pd.DataFrame(arff.loadarff('voting_with_missing.arff')[0])
x_train = data.drop(columns="Class")
y_train = data["Class"]
dt = DTClassifier()
dt.fit(x_train, y_train, True)
print(dt.ig_splits)

0.9302325581395349
0.9090909090909091
0.9534883720930233
0.9318181818181818
0.9767441860465116
0.8863636363636364
0.9302325581395349
0.8863636363636364
0.9767441860465116
0.9772727272727273
average accuracy: 0.9358350951374206
physician-fee-freeze = ?
    mx-missile = ?
        prediction = republican
    mx-missile = n
        prediction = democrat
    mx-missile = y
        anti-satellite-test-ban = ?
            prediction = democrat
        anti-satellite-test-ban = n
            prediction = republican
        anti-satellite-test-ban = y
            prediction = democrat
physician-fee-freeze = n
    adoption-of-the-budget-resolution = ?
        prediction = democrat
    adoption-of-the-budget-resolution = n
        education-spending = ?
            prediction = republican
        education-spending = n
            synfuels-corporation-cutback = n
                religious-groups-in-schools = n
                    crime = n
                        prediction = democrat
           

For the 10 fold cross validation, the overall accuracy was about 94% or .94. Which is about the threshold it should be. It seems it did better on the voting datadet than it did with the cars dataset. Although there was one fold that got 1.0 as the test clasification accuracy.

### 2.4 (5%) Decision Tree Intuition
- For each of the two problems above, summarize in English what the decision tree has learned (i.e. look at the induced tree and describe what "rules" it has discovered to try to solve each task). 
- If the tree is very large you can just discuss a few of the more shallow attribute combinations and the most important decisions made high in the tree.

In the Cars dataset, The Tree learned that the thing that most people want number 1 in a car is safety. It found that it best split with safety and then split at the amount of people it could hold. then it would split by buying and maintenence. It appears that safety is the one that people care about, in fact, that for a low safety rating, it automatically predicted that it was automatically in the "unacc" class.


In the voting dataset, The tree leanred that depending on if they voted for physician fee freeze it could predic thier class more accuratly. For those who were unsure of a physician fee freeze the next split to determine was mx missile. If it was y on a physician fee freeze, the next split would be by synfuels corporate cutback. If it was n on the physician fee freeze, the next split would be adoption of the budget resolution. 

## 3 Using SciKit Learn's decision tree  

### 3.1 (10%) SK Learn on Voting Dataset
- Use SciKit learns decision tree (CART) on the voting dataset and compare the results with your ID3 version. Use this [Voting Dataset with missing values].(https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/voting_with_missing.arff)
- Try different parameters and report what parameters perform the best on the test set.

In [25]:
def convert( data):
    if isinstance(data, bytes):  return data.decode()
    if isinstance(data, dict):   return dict(map(convert, data.items()))
    if isinstance(data, tuple):  return tuple(map(convert, data))
    if isinstance(data, list):   return list(map(convert, data))
    return data
data = arff.loadarff('voting_with_missing.arff')[0]
data = convert(data)
df =  pd.DataFrame(data, dtype=float)

clf = tree.DecisionTreeClassifier()
X = df.drop(columns="Class").values
y = df["Class"].values
X = np.array(convert(X.tolist()))
y = np.array(convert(y.tolist()))

clf = clf.fit(X, y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  if __name__ == '__main__':


ValueError: ignored

Discuss scikit CART results & also compare to your ID3 results

### 3.2 (10%) Choose a data set of your choice (not already used in this or previous labs) and use the SK decision tree to learn it. Experiment with different hyper-parameters to try to get the best results possible.

NameError: ignored

Discussion

### 3.3 (5%) Print sklearn's decision tree for your chosen data set (using export_graphviz or another tool) and discuss what you find. If your tree is too deep to reasonably fit on one page, show only the first several levels (e.g. top 5).

NameError: ignored

Discussion

## 4. (Optional 5% extra credit) Implement reduced error pruning to help avoid overfitting
- You will need to take a validation set out of your training data to do this, while still having a test set to test your final accuracy. 
- Create a table comparing your decision tree implementation's results on the cars and voting data sets with and without reduced error pruning. 
- This table should compare:
    - a) The # of nodes (including leaf nodes) and tree depth of the final decision trees 
    - b) The generalization (test set) accuracy. (For the unpruned 10-fold CV models, just use their average values in the table).

In [ ]:
# Reduced Error Pruning Code

Discussion